In [1]:
%pylab
%matplotlib inline
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


# 第9章 数据聚合与分组运算


对数据集进行分组并对各组应用一个函数（无论是聚合还是转换），这是数据分析工作中的重要环节。在数据集准备好后，通常任务就是**计算分组统计**或**生成透视表**。

pandas提供了一个灵活高效的groupby功能，以一种自然方式对数据集进行切片、切块、摘要等操作。

本章内容：
- 根据一个或多个键（可以是函数、数组或DataFrame列名）拆分pandas对象。
- 计算分组摘要统计，如技术、平均值、标准拆，或用户自定义函数。
- 对DataFrame的列应用各种函数
- 应用组内转换或其他运算，如规格化、线性回归、排名或选取子集等
- 计算透视表或交叉表
- 执行分位数分析以及其他分组分析

## GroupBy技术
分组运算术语“split-apply-combine”拆分-应用-合并。
1. pandas对象中的数据根据提供的一个或多个键被**拆分**为多组。
2. 将一个函数**应用**到各个分组并产生一个新值。
3. 所有结果被**合并**到最终的结果对象中。

分组键可以有多种形式，且类型不必相同：
- 列表或数组，其长度与待分组的轴一样
- 表示DataFrame某个列名的值
- 字典或Series，给出待分组轴上的值与分组名之间的对应关系
- 函数，用于处理轴索引或索引中的各个标签

In [3]:
# 以表格型数据集为例
df = DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'],
               'key2': ['one', 'two', 'one', 'two', 'one'],
               'data1': np.random.randn(5),
               'data2': np.random.randn(5)})
df

,data1,data2,key1,key2
0,-0.644547,0.182089,a,one
1,-1.198290,1.124191,a,two
2,0.068472,0.213797,b,one
3,0.715427,-0.365413,b,two
4,-0.068832,-1.112716,a,one


In [5]:
# 按key1分组，并计算data1列的平均值
# 访问data1，根据key1调用groupby
grouped = df['data1'].groupby(df['key1'])
grouped
# grouped是一个GroupBy对象，但没有进行任何计算，只含有一些有关分组键df['key1']的中间数据
# 换句话说，该对象已经有了接下来对各分组执行运算所需的一切信息。

In [6]:
# 调用GroupBy对象的mean方法来计算平均值
grouped.mean()

key1
a   -0.637223
b    0.391950
Name: data1, dtype: float64

In [9]:
# 一次传入多个数组：
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one    -0.356690
      two    -1.198290
b     one     0.068472
      two     0.715427
Name: data1, dtype: float64

In [10]:
# unstack()行-》列，默认操作最内级
means.unstack()

key2,one,two
key1,,
a,-0.356690,-1.198290
b,0.068472,0.715427


In [12]:
# 分组键不仅为Series，可以为任何长度适当的数组：
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])

df['data1'].groupby([states, years]).mean()

California  2005   -1.198290
            2006    0.068472
Ohio        2005    0.035440
            2006   -0.068832
Name: data1, dtype: float64

In [13]:
# GroupBy的size方法，返回一个含有分组大小的Series：
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### 对分组进行迭代
GroupBy对象支持迭代，可以产生一组二元元组（由分组名和数据块组成）

for name, group in df.groupby('key1'):
    print name
    print group

对于多重键的情况，元组的第一个元素将会是由键值组成的元组：

In [15]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print k1, k2
    print group

a one
      data1     data2 key1 key2
0 -0.644547  0.182089    a  one
4 -0.068832 -1.112716    a  one
a two
     data1     data2 key1 key2
1 -1.19829  1.124191    a  two
b one
      data1     data2 key1 key2
2  0.068472  0.213797    b  one
b two
      data1     data2 key1 key2
3  0.715427 -0.365413    b  two


In [17]:
# 将这些数据片段做成一个字典
pieces = dict(list(df.groupby('key1')))
pieces['a']

,data1,data2,key1,key2
0,-0.644547,0.182089,a,one
1,-1.198290,1.124191,a,two
4,-0.068832,-1.112716,a,one


groupby默认在axis=0上进行分组，设置后可以在其他任何轴上分组。根据dtype对列进行分组：

In [19]:
df.dtypes

data1    float64
data2    float64
key1      object
key2      object
dtype: object

In [20]:
grouped = df.groupby(df.dtypes, axis=1)

In [22]:
dict(list(grouped))

{dtype('float64'):       data1     data2
 0 -0.644547  0.182089
 1 -1.198290  1.124191
 2  0.068472  0.213797
 3  0.715427 -0.365413
 4 -0.068832 -1.112716, dtype('O'):   key1 key2
 0    a  one
 1    a  two
 2    b  one
 3    b  two
 4    a  one}

### 选择一个或一组列
用一个或一组列名对DataFrame产生的GroupBy对象进行索引，就行实现选取部分列进行聚合的目的。
```
df.groupby('key1')['data1']
df.groupby('key1')[['data2']]
```
是一下代码的语法糖：
```
df['data1'].groupby(df['key1'])
df[['data2']].groupby(df['key1'])
```

In [24]:
# 计算data2列的平均值并以DataFrame形式得到结果：
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one  -0.465313
     two   1.124191
b    one   0.213797
     two  -0.365413

### 通过字典或Series进行分组
除数组外，分组信息还可以其他形式存在。

In [25]:
people = DataFrame(np.random.randn(5, 5),
                  columns=['a', 'b', 'c', 'd', 'e'],
                  index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

people.ix[2:3, ['b', 'c']] = np.nan # 添加几个NA值
people

,a,b,c,d,e
Joe,0.677087,-0.073022,0.127327,0.240203,-1.401573
Steve,1.865376,0.394724,-0.299114,-1.101817,0.617375
Wes,-1.049438,NaN,NaN,1.623710,0.433252
Jim,-2.136730,-0.393873,0.192103,0.757828,0.017963
Travis,-0.087962,0.639583,-0.655640,-0.392498,0.243077


In [26]:
# 已知列的分组关系，并希望根据分组计算列的总计：
mapping = {'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f': 'orange'}

# 将字典传给groupby
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,0.367530,-0.797509
Steve,-1.400932,2.877475
Wes,1.623710,-0.616186
Jim,0.949932,-2.512641
Travis,-1.048138,0.794698


Series也有同样的功能，它可以被看做一个固定大小的映射。对于上面那个例子，如果Series作为分组键，则pandas会检查Series以确保其索引跟分组轴是对齐的：

In [27]:
map_series = Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [28]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### 通过函数进行分组
任何被当做分组键的函数都会在各个索引值上被调用一次，其返回值就会被用作分组名称。

以上节为例，其索引值为人的名字，希望根据人名长度进行分组，虽然可以求取一个字符串长度数组，但其实仅仅传入len函数就可以了：

In [29]:
people.groupby(len).sum()

,a,b,c,d,e
3,-2.509081,-0.466895,0.319430,2.621741,-0.950359
5,1.865376,0.394724,-0.299114,-1.101817,0.617375
6,-0.087962,0.639583,-0.655640,-0.392498,0.243077


将函数跟数组、列表、字典、Series混合使用也不是问题，因为任何东西最终都会被转换为数组：

In [31]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -1.049438 -0.073022  0.127327  0.240203 -1.401573
  two -2.136730 -0.393873  0.192103  0.757828  0.017963
5 one  1.865376  0.394724 -0.299114 -1.101817  0.617375
6 two -0.087962  0.639583 -0.655640 -0.392498  0.243077

### 根据索引级别分组
通过level关键字传入级别编号或名称即可：

In [32]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]], names=['cty', 'tenor'])

In [33]:
hier_df = DataFrame(np.random.randn(4,5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.206773 -0.143103 -1.702116  1.301787 -0.449864
1     -0.188117 -0.117897  0.160675 -0.596424 -1.584560
2      0.596048 -0.517599  0.466526  0.190777  0.192368
3     -1.212000 -0.335916 -0.066398 -1.142919  0.961317

In [34]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 数据聚合
聚合：任何能够从数组产生标量值得数据转换过程。之前的例子比如mean、count、min以及sum等。

**经过优化的GroupBy的方法**

**函数名** | **说明**
- | -
count | 分组中非NA值的数量
sum | 非NA值的和
mean | 非NA值得平均值
median | 非NA值得算数中位数
std、var | 无偏（分母n-1）标准差和方差
min、max | 非NA值得最小和最大值
prod | 非NA值的积
first、last | 第一个和最后一个非NA值

也可以使用自己发明的聚合运算。

运行方式，例如quantile，没有明确地实现于GroupBy，但它是一个Series方法，所以这里是能用的。GroupBy会高效地对Seris进行切片，然后对各片调用piece.quantile()，最后将这些结果组装成最终结果。

如果要使用自己的聚合函数，只需将其传入aggregate或agg方法即可：

In [36]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

grouped = df.groupby('key1')
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.129457,2.236907
b,0.646955,0.579210


In [37]:
# 有些方法也可以在这里用，即使不是聚合运算：
grouped.describe()

data1     data2
key1                          
a    count  3.000000  3.000000
     mean  -0.637223  0.064522
     std    0.564764  1.123078
     min   -1.198290 -1.112716
     25%   -0.921418 -0.465313
     50%   -0.644547  0.182089
     75%   -0.356690  0.653140
     max   -0.068832  1.124191
b    count  2.000000  2.000000
     mean   0.391950 -0.075808
     std    0.457467  0.409564
     min    0.068472 -0.365413
     25%    0.230211 -0.220610
     50%    0.391950 -0.075808
     75%    0.553688  0.068995
     max    0.715427  0.213797

**注意：**自定义的聚合函数要比优化过的表中的函数慢很多，因为在构造中间分组数据块时存在非常大的开销（函数调用、数据重排等）。

### 面向列的多函数应用
对不同的列使用不同的聚合函数，或一次应用多个函数。首先根据sex和smoker对tips进行分组：

In [38]:
tips = pd.read_csv('old-file/ch08/tips.csv')

# 添加“消费占总额百分比”的列
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:5]

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808


In [39]:
grouped = tips.groupby(['sex', 'smoker']) # 多层次索引的DataFrame
grouped_pct = grouped['tip_pct'] # 多层次索引的Series

grouped_pct.agg('mean')

sex     smoker
Female  No        0.156921
        Yes       0.182150
Male    No        0.160669
        Yes       0.152771
Name: tip_pct, dtype: float64

In [40]:
# 传入一组函数或函数名，的到的DataFrame的列就会以相应的函数命名：
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
sex    smoker                                  
Female No      0.156921  0.036421      0.195876
       Yes     0.182150  0.071595      0.360233
Male   No      0.160669  0.041849      0.220186
       Yes     0.152771  0.090588      0.674707

不一定接受GroupBy自动给出的列名，特别是lambda函数，名称为'<lambda>',如果传入的是一个由***(name, function)***元组组成的列表，则各元组的第一个元素就会被用作DataFrame的列名（可以将这种二元元组列表看做一个有序映射）：

In [42]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std), ('ptp', peak_to_peak)])

foo       bar       ptp
sex    smoker                              
Female No      0.156921  0.036421  0.195876
       Yes     0.182150  0.071595  0.360233
Male   No      0.160669  0.041849  0.220186
       Yes     0.152771  0.090588  0.674707

In [43]:
# 一组应用于全部列的函数，或不同的列应用不同的函数。
functions = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(functions)

result

tip_pct                     total_bill                  
                count      mean       max      count       mean    max
sex    smoker                                                         
Female No          54  0.156921  0.252672         54  18.105185  35.83
       Yes         33  0.182150  0.416667         33  17.977879  44.30
Male   No          97  0.160669  0.291990         97  19.791237  48.33
       Yes         60  0.152771  0.710345         60  22.284500  50.81

In [44]:
# DataFrame拥有层次化的列，这相当于分别对各列进行聚合，然后用concat将结果组装到一起
result['tip_pct']

count      mean       max
sex    smoker                           
Female No         54  0.156921  0.252672
       Yes        33  0.182150  0.416667
Male   No         97  0.160669  0.291990
       Yes        60  0.152771  0.710345

In [46]:
# 可以传入带有自定义名称的元组列表
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]
grouped['tip_pct', 'total_bill'].agg(ftuples)

tip_pct              total_bill           
              Durchschnitt Abweichung Durchschnitt Abweichung
sex    smoker                                                
Female No         0.156921   0.001327    18.105185  53.092422
       Yes        0.182150   0.005126    17.977879  84.451517
Male   No         0.160669   0.001751    19.791237  76.152961
       Yes        0.152771   0.008206    22.284500  98.244673

In [47]:
# 不用的列应用不同的函数，向agg传入一个从列名映射到函数的字典：
grouped.agg({'tip': np.max, 'size': 'sum'})

tip  size
sex    smoker            
Female No       5.2   140
       Yes      6.5    74
Male   No       9.0   263
       Yes     10.0   150

In [48]:
grouped.agg({'tip_pct': ['min', 'max', 'mean', 'std'], # 对tip_pct这列应用四个不同的函数，返回四个结果
            'size': 'sum'})

tip_pct                               size
                    min       max      mean       std  sum
sex    smoker                                             
Female No      0.056797  0.252672  0.156921  0.036421  140
       Yes     0.056433  0.416667  0.182150  0.071595   74
Male   No      0.071804  0.291990  0.160669  0.041849  263
       Yes     0.035638  0.710345  0.152771  0.090588  150

### 以“无索引”的形式返回聚合数据